In [1]:
import plotly 
plotly.tools.set_credentials_file(username='tutorial.account', api_key='gxa2ckhtj2')

In [2]:
from math import sin
from numpy import linspace

xdata = [x for x in linspace(-10,10,100)]

# y1(x) = x^2
y1_data = [x**2 for x in xdata]

trace1 = {
    'type': 'scatter',
    'mode': 'line',
    'x': xdata,
    'y': y1_data
}

# y2(x) = 10sin(x)
y2_data = [10*sin(x) for x in xdata]

trace2 = {
    'type': 'scatter',
    'mode': 'line',
    'line': {'dash': 'dash'},
    'x': xdata,
    'y': y2_data
}

data_traces = [trace1, trace2]

layout = {
    'title': 'Sine waves and Parabolas',
    'xaxis': {'title': 'x'},
    'yaxis': {'title': 'y(x)'}
}


from plotly.plotly import iplot

# put the figure specification together
figure = {
    'data': data_traces, 
    'layout': layout
}

# visualize the figure
iplot(
    figure,
    filename="waves-and-parabolas"
)

High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~tutorial.account/0 or inside your plot.ly account where it is named 'waves-and-parabolas'


In [3]:
from numpy import linspace, meshgrid
from plotly.graph_objs import Heatmap

grid_points = linspace(-10,10,200)

# X and Y are each 100 by 100 matrices
x, y = meshgrid(grid_points, grid_points)

zdata = x*x + y*y

# Can use constructors in plotly.graph_objs
# to generate plot traces
trace = Heatmap(z=zdata)

figure = {
    'data': [trace],
    'layout': {
        # You can even use mathjax syntax
        # for latex mode in strings
        'title': '$$Z = X^2 + Y^2$$'
    }
}

from plotly.plotly import iplot

iplot(figure, filename="heatmap-blob")

In [4]:
# Keeping everything 
# from the heatmap example
# but changing the trace's 
# type to surface

from plotly.graph_objs import Surface

trace = Surface(z=zdata)

figure = {
    'data': [trace],
    'layout': {
        'title': '$$X^2 + Y^2$$'
    }
}

iplot(figure, filename="surface-bowl")

In [5]:
from numpy import linspace, meshgrid, gradient, exp

x_range = linspace(-2,2,20)
y_range = linspace(-2,2,16)

x,y = meshgrid(x_range, y_range)

z = x*exp(-x**2 - y**2)
v, u = gradient(z)

from plotly.tools import FigureFactory

# Create quiver figure
figure = FigureFactory.create_quiver(
    x=x,
    y=y,
    u=u,
    v=v,
    scale=0.8,
    arrow_scale=0.6,
    name='quiver',
    line=dict(width=1, color='black')
)
from plotly.plotly import iplot

iplot(figure, filename='function-gradient-quiver')

In [6]:
from plotly.graph_objs import Scatter, Heatmap

# Create points
points = Scatter(
    x=[-.7, .75], y=[0,0],
    mode='markers',
    marker=dict(size=12),
    name='points'
)

heatmap = Heatmap(
    x=x_range,
    y=y_range,
    z=z,
    name="heatmap"
)

# Add points to figure
figure['data'].append(points)

# Adds heatmap underlay
figure['data'] = [heatmap] + figure['data']

# Set figure title and hide legend
figure['layout']['title'] = "$$xe^{-x^2 - y^2}$$"
figure['layout']['showlegend'] = False

from plotly.plotly import iplot

iplot(figure, filename='function-gradient')

In [7]:
from pandas import read_csv

data_frame = read_csv(
    'https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv'
)

In [8]:
# Countries we're interested in.
countries = ['BRA', 'CAN', 'CHE', 'RUS', 'BEL']

# Subset of the data frame involving the above contries
subset = data_frame[data_frame['CODE'].isin(countries)]

# Create a dictionary mapping country code to GDP
gdp = {}

for index, row in subset.iterrows():
    gdp[row['CODE']] = row['GDP (BILLIONS)']

from random import random
from copy import deepcopy

def update_gdp(gdp):
    copy_gdp = deepcopy(gdp)
    for key in copy_gdp:
        # Random gaussian growth rate between -0.20 and 0.20
        value = copy_gdp[key]
        growth_rate = (random()*2 - 1) * 0.2
        copy_gdp[key] = value + value * growth_rate

    return copy_gdp


# Container for sequences of GDP updates
gdp_sequences = {}

# Append a gdp update to the sequences
def append_gdp(gdp):
    for key in countries:
        gdp_sequences[key] = gdp_sequences.get(key, []) + [gdp[key]]

# Initialize the sequence with some GDPs for each country
append_gdp(gdp)

# Generate a sequence of GDPs for each country
for i in range(20):
    append_gdp(update_gdp(gdp))

    
from plotly.graph_objs import Scatter

def make_line(key):
    return Scatter(
        y=gdp_sequences[key], 
        name=key,
        mode="line"
    )

# Generate Line traces to plot in the figure
line_traces = [make_line(key) for key in countries]

layout = {
    'title': 'GDP Snapshot',
    'xaxis': {'title': 'Time (years)'},
    'yaxis': {'title': 'GDP (BILLIONS)'}
}

figure_lines = {
    'data': line_traces,
    'layout': layout
}


from plotly.plotly import iplot

iplot(figure_lines, filename="gdp-snapshot")

In [9]:
from plotly.graph_objs import Bar

# Generate Line traces to plot in the figure
gdp_update = update_gdp(gdp)

bar_trace = Bar(
    x=countries,
    y=[gdp_update[key] for key in countries],
    name="GDP bars"
)

layout = {
    'title': 'GDP for One Year',
    'xaxis': {'title': 'Country Code'},
    'yaxis': {'title': 'GDP (BILLIONS)'}
}

figure_bar = {
    # Plot a single bar chart
    'data': [bar_trace],
    'layout': layout
}

from plotly.plotly import iplot

iplot(figure_bar, filename="gdp-bars")

In [10]:
from plotly.graph_objs import Choropleth

# Generate Line traces to plot in the figure
gdp_update = update_gdp(gdp)

choro_trace = Choropleth(
    locations=countries,
    z=[gdp_update[key] for key in countries],
    name="GDP choropleth",

    # Configure the legend's color bar
    colorbar = {
        'tickprefix': '$',
        'ticksuffix': 'B',
        'title': 'GDP<br>Billions US$'
    }
)

layout = {
    'title': 'Map for GDP for One Year'
}

figure_choro = {
    # Plot a single bar chart
    'data': [choro_trace],
    'layout': layout
}

from plotly.plotly import iplot

iplot(figure_choro, filename="gdp-choropleth")

In [11]:
from plotly.tools import make_subplots

full_figure = make_subplots(
    rows = 2,
    cols = 2,
    subplot_titles = (
        'GDP Time Series',
        'GDP for One Year',
    )
)

# Hide choropleth color bar
choro_trace['showscale'] = False


# Set traces into their subplots    
bar_trace['xaxis'] = 'x2'
bar_trace['yaxis'] = 'y2'

choro_trace['geo'] = 'geo3'


# Accumulate traces into one array
full_traces = [choro_trace, bar_trace] + line_traces

# Set the traces of the sub plot figure
full_figure['data'] = full_traces
full_figure['layout']['title'] = 'GDP Sub Plot'

full_figure['layout']['geo3'] = {
    # set the position and size using relative viewspace positions
    'domain': {'x': [0.0, 1.0], 'y': [0.0, 0.35]}
}

from plotly.plotly import iplot

iplot(full_figure, filename="gdp-sub-plot")

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



In [12]:
streaming_keys = [
    'kqrbk2g9jo', 'brb0um2l0c', 'osv8zy33nh', 
    'q4wdes5yy0', 'z008j82eji', '51kncwir1s', 
    'c3ymqy6g1j'
]


MAX_POINTS = 60


from plotly.graph_objs import Stream
from plotly.plotly import Stream as plotlyStream

stream_writers = {}

i = 0
stream_writers['lines'] = {}
for line in line_traces:
    stream = Stream(token=streaming_keys[i], maxpoints=MAX_POINTS)
    stream_writers['lines'][line['name']] = plotlyStream(streaming_keys[i])
    line['stream'] = stream
    i += 1

stream = Stream(token=streaming_keys[i], maxpoints=len(countries))
stream_writers['bar'] = plotlyStream(streaming_keys[i])
bar_trace['stream'] = stream

i += 1

stream = Stream(token=streaming_keys[i], maxpoints=len(countries))
stream_writers['choro'] = plotlyStream(streaming_keys[i])
choro_trace['stream'] = stream

In [42]:
def init_streams():
    for line in line_traces:
        stream_writers['lines'][line['name']].open()
    stream_writers['bar'].open()
    stream_writers['choro'].open()
    
def close_streams():
    for line in line_traces:
        stream_writers['lines'][line['name']].close()
    stream_writers['bar'].close()
    stream_writers['choro'].close()


from time import sleep    

def update_streams(streamed_gdp):
    for line in line_traces:
        stream_writers['lines'][line['name']].write({
            'type': 'scatter',
            'mode': 'line',
            'y': streamed_gdp[line['name']]
        })
        # The server won't recieve data 
        # faster than 50 miliseconds at a time.
        sleep(0.06)
        
    for country in countries:
        stream_writers['bar'].write({
            'type': 'bar',
            'x': country,
            'y': streamed_gdp[country]
        })
        sleep(0.06)
        
        stream_writers['choro'].write({
            'type': 'choropleth',
            'locations': country,
            'z': streamed_gdp[country]
        })
        sleep(0.06)
    
    # Use our fake data generator 
    # from before to update GDPs
    return update_gdp(streamed_gdp)

# Accumulate traces into one array
full_traces = [choro_trace, bar_trace] + line_traces

# Set the traces of the sub plot figure
full_figure['data'] = full_traces
full_figure['layout']['title'] = 'GDP Streaming Sub Plot'

full_figure['layout']['geo3'] = {
    # set the position and size using relative viewspace positions
    'domain': {'x': [0.0, 1.0], 'y': [0.0, 0.35]}
}

from plotly.plotly import plot

iplot(full_figure, filename="gdp-sub-plot-streaming")

In [45]:
init_streams()

streamed_gdp = gdp
for i in range(15):
    streamed_gdp = update_streams(streamed_gdp)
    sleep(0.5)

close_streams()

In [15]:
from numpy import linspace, meshgrid, zeros

x_range = linspace(-2,2,200)
y_range = linspace(-2,2,200)

x, y = meshgrid(x_range, y_range)

c = x + y*1j

def f(z,c):
    return z**2 + c

NUM_ITERATIONS = 40
def mandelbrot(c):
    z = zeros(c.shape)
    iter_matrix = zeros(c.shape)
    for i in range(NUM_ITERATIONS):
        iter_matrix[z<2.0] = i
        z = f(z,c)
    iter_matrix = iter_matrix / float(NUM_ITERATIONS)
    return iter_matrix

mandelbrot_set = mandelbrot(c)


colorscale=[
    [0.0, 'rgb(20,29,67)'],
    [0.1, 'rgb(28,76,96)'],
    [0.2, 'rgb(16,125,121)'],
    [0.3, 'rgb(92,166,133)'],
    [0.4, 'rgb(182,202,175)'],
    [0.5, 'rgb(253,245,243)'],
    [0.6, 'rgb(230,183,162)'],
    [0.7, 'rgb(211,118,105)'],
    [0.8, 'rgb(174,63,95)'],
    [0.9, 'rgb(116,25,93)'],
    [1.0, 'rgb(51,13,53)']
]

from plotly.graph_objs import Surface

# 3D surface of mandelbrot 
# set offset 1.0 units upward
popout_layer = Surface(
    x=x,
    y=y,
    z=mandelbrot_set + 1.0,
    colorscale=colorscale
)

#Flat projection of mandelbrot set under 3d surface
flat_layer = Surface(
    x=x,
    y=y,
    z=zeros(mandelbrot_set.shape), # Make the projection flat
    surfacecolor=mandelbrot_set,
    colorscale=colorscale
)

data = [popout_layer, flat_layer]

layout = {
    'title': 'Mandelbrot Surface'
}

figure = {
    'data': data,
    'layout': layout
}


from plotly.plotly import iplot

iplot(figure, filename="fractal-surface")